# Computing the graph parameters for the activities

- degree distributions, clustering coefficient distributions, average degree, and average clustering coeff
- The degree distributions are used in the main manuscript. 


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
from scipy.sparse import csr_matrix
import time


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
from sklearn.preprocessing import normalize

In [3]:
import sys
sys.path.append("../")
import utils

In [4]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [5]:
# read files
with open("../../data/02_intermediate/boston_stays.pickle", 'rb') as f:
    df_boston = pickle.load(f)

with open("../../data/02_intermediate/miami_stays.pickle", 'rb') as f:
    df_miami = pickle.load(f)

with open("../../data/02_intermediate/chicago_stays.pickle", 'rb') as f:
    df_chicago = pickle.load(f)

with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)
    
with open("../../data/03_processed/A_home_activity_three_cities_weighted_dic.pickle", 'rb') as f:
    A_home_activity_weighted_dic = pickle.load(f)
    

In [6]:
# read the spatial network dictionary.
with open("../../data/03_processed/spatial_network_boston_miami_chicago_dic.pickle", 'rb') as f:
    spatial_network_dic = pickle.load(f)
    

In [7]:
# read shapefiles
with open("../../data/02_intermediate/boston_miami_chicago_ct_shp_dic.pickle", 'rb') as f:
    shp_dic = pickle.load(f)


In [8]:
# read evaluation files
with open("../../data/05_model_outputs/lasso_coefficients.pickle", 'rb') as f:
    lasso_coef = pickle.load(f)

In [9]:
# check the coefficients
lasso_coef['boston']['inc_median_household_2018']['lasso (no socio-demographics)'].index


Index(['Latin American', 'Caribbean', 'Brazilian', 'Fried Chicken',
       'Laundromat', 'Food Stand', 'Tennis Court', 'Fishing Store', 'Football',
       'Science Museum', 'Ski Area', 'Peking Duck', 'Cupcakes', 'Skating Rink',
       'French'],
      dtype='object')

In [10]:
# check the mobility network
A_home_activity_unweighted_dic['boston'][1.0]['ATM']


GEOID,25009211402,25017357500,25025101102,25017374200,25025020301,25017316102,25017351300,25017330200,25009260900,25017374700,...,25025060501,25017333100,25017333400,25017336300,25025110301,25021443101,25021400400,25017339400,25009268200,25009221700
GEOID_home,,,,,,,,,,,,,,,,,,,,,
25009201100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009202101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009202102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009202200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25009203100,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33017085000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33017086000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33017087000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Turn the directed to undirected adj matrice (redundant...)

- Slow. Consider reading the A_home_activity_unweighted_sym_adj_dic directly.

In [12]:
def turn_asymmetric_directed_to_symmetric_undirected_adj(activity_dir_adj_df):
    ''' turn an asymmetric directed adj matrix to a symmetric directed one '''
#     activity_dir_adj_df = A_home_activity_unweighted_dic['boston'][1.0]['ATM']
    union_list = sorted(set(activity_dir_adj_df.columns).union(set(activity_dir_adj_df.index)))
    activity_zeros_df = pd.DataFrame(np.zeros((len(union_list), len(union_list))), 
                                     columns = union_list,
                                     index = union_list)
    activity_zeros_df = activity_zeros_df.add(activity_dir_adj_df, fill_value = 0.0)
#     print(activity_zeros_df)

    activity_undir_adj_df = pd.DataFrame(np.maximum(activity_zeros_df.values.T, activity_zeros_df.values),
                                   columns = union_list,
                                   index = union_list)
    return activity_undir_adj_df



In [13]:
A_home_activity_unweighted_sym_adj_dic = {}

for city in ['boston','chicago','miami']:
    print(city)
    A_home_activity_unweighted_sym_adj_dic[city] = {}
    for threshold in [1.0, 10.0]:
        print(threshold)
        A_home_activity_unweighted_sym_adj_dic[city][threshold] = {}
        for activity_name in A_home_activity_unweighted_dic[city][threshold].keys():
            A_home_activity_unweighted_sym_adj_dic[city][threshold][activity_name]=turn_asymmetric_directed_to_symmetric_undirected_adj(A_home_activity_unweighted_dic[city][threshold][activity_name])
            

boston
1.0
10.0
chicago
1.0
10.0
miami
1.0
10.0


### Compute the graph parameters for activity categories

In [33]:
###
beginning_time = time.time()
mobility_network_parameters_dic = {}
threshold = 1.0

for city in ['boston','chicago','miami']:
    print(city)
    mobility_network_parameters_dic[city]={}
    for idx in range(len(A_home_activity_unweighted_sym_adj_dic[city][threshold].keys())):
        activity_name = list(A_home_activity_unweighted_sym_adj_dic[city][threshold].keys())[idx]
        mobility_network_parameters_dic[city][activity_name] = {}
        current_time = time.time()
        elapse_time = current_time - beginning_time
        print(idx, activity_name, elapse_time/60.0, "minutes", end = '\r')

        # init the mobility graph
        adj_m = A_home_activity_unweighted_sym_adj_dic[city][threshold][activity_name]
        G_mobility = nx.from_pandas_adjacency(adj_m)

        # computing the parameters
        # degrees, degree distributions, avearge degree, clustering coeff distributions. 
        degrees = nx.degree(G_mobility)
        degree_distributions = np.unique(list(dict(nx.degree(G_mobility)).values()), return_counts = True)
        average_degree = np.mean(list(dict(degrees).values()))
        clustering = nx.clustering(G_mobility)
        average_clustering = np.mean(list(clustering.values()))        
        
        # save
        mobility_network_parameters_dic[city][activity_name]['degrees']=degrees
        mobility_network_parameters_dic[city][activity_name]['degree_distributions']=degree_distributions
        mobility_network_parameters_dic[city][activity_name]['average_degree']=average_degree
        mobility_network_parameters_dic[city][activity_name]['clustering']=clustering
        mobility_network_parameters_dic[city][activity_name]['average_clustering']=average_clustering        


boston
chicago Exhibit 0.5304084300994873 minutesestesesnutess
miamioo Exhibit 3.004456639289856 minutestesutessnutess


## Save!

In [34]:
# save!
with open('../../data/05_model_outputs/network_property_degree_clustering.pickle', 'wb') as f:
    pickle.dump(mobility_network_parameters_dic, f)
    